In [2]:
import cdsapi
import findspark
findspark.init('/home/thiagocaminha/spark-3.5.1-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
import xarray as xr
import pandas

In [7]:
c = cdsapi.Client()


In [8]:
type(c)

cdsapi.api.Client

In [31]:
# EXTRACT
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'year': '2023',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_wave_direction',
            'mean_wave_period', 'sea_surface_temperature', 'significant_height_of_combined_wind_waves_and_swell',
        ],
        'month': '07',
        'day': '30',
        'time': '12:00',
        'area': [
            -19, -48, -27,
            -36,
        ],
    },
    'data.nc')

# ds = xr.load_dataset("data.grib")

df = ds.to_dataframe()


2024-04-29 12:59:10,617 INFO Welcome to the CDS
2024-04-29 12:59:10,617 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-29 12:59:11,742 INFO Request is queued
2024-04-29 13:18:09,336 WARNING Connection error: [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/v2/tasks/02871c8b-8f4b-404e-bc68-90e0c822df82 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77361884a460>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno -3] Temporary failure in name resolution)"))]. Attempt 1 of 500.
2024-04-29 13:18:09,337 WARNING Retrying in 120 seconds
2024-04-29 13:20:09,437 INFO Retrying now...
2024-04-29 13:24:13,170 INFO Request is completed
2024-04-29 13:24:13,172 INFO Downloading https://download-0012-clone.copernicus-climate.eu/cache-compute-0012/cache/data5/adaptor.mars.internal-1714407826.3642888-6281-3-02871c8b-8f4b-404e-bc68-90e0c822df82.nc to 

In [42]:
ds = xr.load_dataset("test_etl.nc")#.to_dataframe()
# df = ds.to_dataframe()


In [36]:
df.to_csv("data.csv")

In [45]:

rename_dict = {'u10':'u_wind_10_meter',
            'v10':'v_wind_10_meter',
            'mwd':'mean_wave_direction',
            'mwp':'mean_wave_period',
            'sst':'sea_surface_temperature',
            'swh':'significant_wave_height_wind_and_swell'
            }

for key in rename_dict:
    

u10
v10
mwd
mwp
sst
swh


In [37]:
df

u10       v10  mwd  mwp  \
longitude latitude time                                                
-48.0     -19.00   2023-07-30 12:00:00 -3.079456  0.418602  NaN  NaN   
          -19.25   2023-07-30 12:00:00 -3.046338  0.024063  NaN  NaN   
          -19.50   2023-07-30 12:00:00 -2.608847 -0.184877  NaN  NaN   
          -19.75   2023-07-30 12:00:00 -1.916427  0.197899  NaN  NaN   
          -20.00   2023-07-30 12:00:00 -1.444756  0.658911  NaN  NaN   
...                                          ...       ...  ...  ...   
-36.0     -26.00   2023-07-30 12:00:00 -0.130278  4.783931  NaN  NaN   
          -26.25   2023-07-30 12:00:00 -0.185908  4.793641  NaN  NaN   
          -26.50   2023-07-30 12:00:00 -0.398878  5.005568  NaN  NaN   
          -26.75   2023-07-30 12:00:00 -0.671368  5.241956  NaN  NaN   
          -27.00   2023-07-30 12:00:00 -0.870549  5.443987  NaN  NaN   

                                               sst  swh  
longitude latitude time                                  
-48.0     -19.00   2023-07-30 12:00:00         NaN  NaN  
          -19.25   2023-07-30 12:00:00         NaN  NaN  
          -19.50   2023-07-30 12:00:00         NaN  NaN  
          -19.75   2023-07-30 12:00:00         NaN  NaN  
          -20.00   2023-07-30 12:00:00         NaN  NaN  
...                                            ...  ...  
-36.0     -26.00   2023-07-30 12:00:00  295.110352  NaN  
          -26.25   2023-07-30 12:00:00  295.150391  NaN  
          -26.50   2023-07-30 12:00:00  295.104492  NaN  
          -26.75   2023-07-30 12:00:00  294.884766  NaN  
          -27.00   2023-07-30 12:00:00  294.474609  NaN  

[1617 rows x 6 columns]

In [9]:
spark = SparkSession.builder.appName("era5-project").getOrCreate()


24/05/16 19:13:15 WARN Utils: Your hostname, thiagocaminha-note resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface wlp0s20f3)
24/05/16 19:13:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/16 19:13:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/05/16 19:13:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [11]:
from pyspark.sql.types import (StructType, StructField,
                                 StringType, IntegerType,
                                 DoubleType, TimestampType)


In [34]:

schema = StructType([
    StructField(name="latitude", dataType=DoubleType(), nullable=True),
    StructField(name="longitude", dataType=DoubleType(), nullable=True),
    StructField(name="time", dataType=TimestampType(), nullable=True),
    StructField(name="u10", dataType=DoubleType(), nullable=True),
    StructField(name="v10", dataType=DoubleType(), nullable=True),
    StructField(name="mwd", dataType=DoubleType(), nullable=True),
    StructField(name="mwp", dataType=DoubleType(), nullable=True),
    StructField(name="sst", dataType=DoubleType(), nullable=True),
    StructField(name="swh", dataType=DoubleType(), nullable=True),
])


data_manual_schema = spark.read.csv("data.csv", header=True, schema=schema)
data = spark.read.csv("data.csv", header=True, inferSchema=True)

In [41]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 49, latitude: 33, time: 1)
Coordinates:
  * longitude  (longitude) float32 -48.0 -47.75 -47.5 ... -36.5 -36.25 -36.0
  * latitude   (latitude) float32 -19.0 -19.25 -19.5 ... -26.5 -26.75 -27.0
  * time       (time) datetime64[ns] 2023-07-30T12:00:00
Data variables:
    u10        (time, latitude, longitude) float32 -3.079 -3.2 ... -0.8705
    v10        (time, latitude, longitude) float32 0.4186 0.4059 ... 5.56 5.444
    mwd        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    mwp        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
    sst        (time, latitude, longitude) float32 nan nan nan ... 294.4 294.5
    swh        (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    Conventions:  CF-1.6
    history:      2024-04-29 16:23:46 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [38]:
data.columns

['longitude', 'latitude', 'time', 'u10', 'v10', 'mwd', 'mwp', 'sst', 'swh']

In [ ]:
{ 'u10':'u_wind_10_meter',
 'v10':'v_wind_10_meter',
 'mwd':'mean_wave_direction',
 'mwp':'mean_wave_period',
 'sst':'sea_surface_temperature', 
 'swh':'significant_wave_height_wind_and_swell'}

In [37]:
data_manual_schema.show()

+--------+---------+-------------------+----------+-----------+---------+--------+----+---------+
|latitude|longitude|               time|       u10|        v10|      mwd|     mwp| sst|      swh|
+--------+---------+-------------------+----------+-----------+---------+--------+----+---------+
|   -48.0|    -19.0|2023-07-30 12:00:00|-3.0794563|   0.418602|     NULL|    NULL|NULL|     NULL|
|   -48.0|   -19.25|2023-07-30 12:00:00| -3.046338|0.024062634|     NULL|    NULL|NULL|     NULL|
|   -48.0|    -19.5|2023-07-30 12:00:00|-2.6088467|-0.18487716|     NULL|    NULL|NULL|     NULL|
|   -48.0|   -19.75|2023-07-30 12:00:00|-1.9164269| 0.19789886|     NULL|    NULL|NULL|     NULL|
|   -48.0|    -20.0|2023-07-30 12:00:00|-1.4447564| 0.65891075|     NULL|    NULL|NULL|     NULL|
|   -48.0|   -20.25|2023-07-30 12:00:00|-1.6204838|   0.983057|     NULL|    NULL|NULL|     NULL|
|   -48.0|    -20.5|2023-07-30 12:00:00|-2.2543278|  1.2859169|     NULL|    NULL|NULL|     NULL|
|   -48.0|   -20.75|

24/05/16 19:25:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: longitude, latitude, time, u10, v10, mwd, mwp, sst, swh
 Schema: latitude, longitude, time, u10, v10, mwd, mwp, sst, swh
Expected: latitude but found: longitude
CSV file: file:///home/thiagocaminha/projects/era5_project/data.csv


In [29]:
data.show()

+---------+--------+-------------------+----------+-----------+---------+--------+----+---------+
|longitude|latitude|               time|       u10|        v10|      mwd|     mwp| sst|      swh|
+---------+--------+-------------------+----------+-----------+---------+--------+----+---------+
|    -48.0|   -19.0|2023-07-30 12:00:00|-3.0794563|   0.418602|     NULL|    NULL|NULL|     NULL|
|    -48.0|  -19.25|2023-07-30 12:00:00| -3.046338|0.024062634|     NULL|    NULL|NULL|     NULL|
|    -48.0|   -19.5|2023-07-30 12:00:00|-2.6088467|-0.18487716|     NULL|    NULL|NULL|     NULL|
|    -48.0|  -19.75|2023-07-30 12:00:00|-1.9164269| 0.19789886|     NULL|    NULL|NULL|     NULL|
|    -48.0|   -20.0|2023-07-30 12:00:00|-1.4447564| 0.65891075|     NULL|    NULL|NULL|     NULL|
|    -48.0|  -20.25|2023-07-30 12:00:00|-1.6204838|   0.983057|     NULL|    NULL|NULL|     NULL|
|    -48.0|   -20.5|2023-07-30 12:00:00|-2.2543278|  1.2859169|     NULL|    NULL|NULL|     NULL|
|    -48.0|  -20.75|

In [ ]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

In [5]:
c = cdsapi.Client()

In [10]:
c.retrieve("reanalysis-era5-single-levels",
{
"variable": "Evaporation",
"pressure_level": "1000",
"product_type": "reanalysis",
"year": "1940",
"month": "01",
"day": "01",
"time": "12:00",
"format": "grib"
}, "download3.grib")

2024-04-10 20:28:14,615 INFO Welcome to the CDS
2024-04-10 20:28:14,617 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2024-04-10 20:28:14,855 INFO Request is queued
2024-04-10 20:28:16,071 INFO Request is completed
2024-04-10 20:28:16,072 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data1/adaptor.mars.internal-1712791694.999173-6413-9-dbb32353-9cd5-4395-9056-2a514aba9376.grib to download3.csv (2M)
2024-04-10 20:28:18,683 INFO Download rate 776.9K/s 


Result(content_length=2076600,content_type=application/x-grib,location=https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data1/adaptor.mars.internal-1712791694.999173-6413-9-dbb32353-9cd5-4395-9056-2a514aba9376.grib)

In [11]:
ds_raw = xr.load_dataset("download3.grib")
ds = ds_raw.copy()

In [ ]:
ds_raw.

In [14]:
ds = ds.rename({"t":"temperature"})
ds["temperature"] = ds["temperature"]

<xarray.Dataset>
Dimensions:        (latitude: 721, longitude: 1440)
Coordinates:
    number         int64 0
    time           datetime64[ns] 1940-01-01T12:00:00
    step           timedelta64[ns] 00:00:00
    isobaricInhPa  float64 1e+03
  * latitude       (latitude) float64 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude      (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time     datetime64[ns] 1940-01-01T12:00:00
Data variables:
    temperature    (latitude, longitude) float32 254.0 254.0 ... 261.6 261.6
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-04-10T20:10 GRIB to CDM+CF via cfgrib-0.9.1...